In [1]:
#!pip install janome

import matplotlib.pyplot as plt
import pandas as pd
import torch
import random
from torch import nn, Tensor
import torch.nn.functional as F
#from torchtext.vocab import vocab
#import torchtext.transforms as T
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
#from torchvision import transforms
import numpy as np
import math
import janome
from janome.tokenizer import Tokenizer
#import spacy
from collections import Counter
from torch.utils.data.sampler import SubsetRandomSampler
import time
#from torchtext.vocab import build_vocab_from_iterator
import levenshtein
import json
import pickle
from timm.scheduler import CosineLRScheduler
import nltk
from nltk import bleu_score
from torch.nn.init import constant_, xavier_uniform_, kaiming_uniform_, xavier_normal_, kaiming_normal_
from torch.nn.parameter import Parameter
from transformers import  get_linear_schedule_with_warmup
from collections import OrderedDict
import gc
import os

In [2]:
with open( "corpus/id_to_word_s.pkl", "rb" ) as f:
    token_list = pickle.load(f)
with open( "corpus/id_to_word_t.pkl", "rb" ) as f:
    token_list_en = pickle.load(f)
with open( "corpus/word_to_id_s.pkl", "rb" ) as f:
    idx_list = pickle.load(f)
with open( "corpus/word_to_id_t.pkl", "rb" ) as f:
    idx_list_en = pickle.load(f)

enc_vocab_size, dec_vocab_size = len(token_list), len(token_list_en)
print(enc_vocab_size, dec_vocab_size)    

49528 20556


In [3]:
a = torch.tensor([ 0, 1,2,3,4,5,6 ])

#print( token_list[2] )

#n = 0

#ii = [ i for i in a ]

#print( ii )

b = [ token_list[i.item()] for i in a ]

print( b )


d = idx_list['<pad>']

print( d )

['<pad>', '<sos>', '<eos>', '<unk>', '<blank>', '<mask>', 'der']
0


In [4]:
# 乱数の種を設定
def random_seed(value):
    torch.backends.cudnn.deterministic=True
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    np.random.seed(value)
    random.seed(value)

# list からランダムに kosuu の数値をとってきて list を作る。
def select( list, kosuu ):
    output = random.sample( list.tolist(), kosuu )
    return output

# ( outer_batch の次元のある)taskset から outer_batch_size の outer_batch データを作る。    
def create_batch_of_tasks(taskset, is_shuffle = True, outer_batch_size = 4):

    idxs = list(range(0,len(taskset)))
    if is_shuffle:
        random.shuffle(idxs)
        idxs = torch.tensor( idxs )
        idxss = select( idxs, outer_batch_size )
    else:
        idxs = torch.tensor( idxs )
        idxss = idxs[ :outer_batch_size ]
    
    output = []

    for i in idxss:
        output.append( taskset[i] )

    return output

# trainset から、num_task を考慮した batch データを作る。
def build_batch_set( trainset, pad_idx_s, pad_idx_t, num_task = 17, k_support = 10, k_query = 10 ):

    count = []
    rr_s = []
    rr_t = []
    rr_len_s = []
    rr_len_t = []
    s_max = []
    t_max = []
    for i in range( num_task ):
        count.append( 0 )
        rr_s.append( [] )
        rr_t.append( [] )
        rr_len_s.append( [] )
        rr_len_t.append( [] )
        s_max.append(0)
        t_max.append(0)
    
    for r in trainset:
        for i in range( num_task ):
            if r['domain'] == i:
                count[i] += 1
                rr_s[i].append( r['source'] )
                rr_t[i].append( r['target'] )
                rr_len_s[i].append( len( r['source']))
                rr_len_t[i].append( len( r['target']))
                if len( r['source'] ) > s_max[i]:
                    s_max[i] = len( r['source'] )
                if len( r['target'] ) > t_max[i]:
                    t_max[i] = len( r['target'] )
    
    ss_max = max( s_max )
    tt_max = max( t_max )
    print( "ss_max:",ss_max )
    print( "tt_max:",tt_max )
                
    #print( rr[0][0] )
    
    for i in range( num_task ):
        print( "i:",i, " count: " ,count[i] )
        
    max_count = max( count )
    min_count = min( count )
    
    #print( "min_count:", min_count )
    
    batch_max = max_count // (k_support + k_query )
    print( "batch_max:", batch_max )
    batch_min = min_count // (k_support + k_query )
    print( "batch_min:", batch_min )
    
    batch = batch_min

    print( batch * ( k_support + k_query ))
    print( len( rr_s[0] ))
    print( len( rr_t[0] ))
    
    output = []

    for i in range( batch ):
        if i % 10000 ==0:
            print( "i:", i )

        output0 = []
        supports_source = []
        supports_target = []
        queries_source = []
        queries_target = []
        supports_source_len = []
        supports_target_len = []
        queries_source_len = []
        queries_target_len = []
        for j in range( num_task ):
            supports_source0 = []
            supports_target0 = []
            queries_source0 = []
            queries_target0 = []
            supports_source0_len = []
            supports_target0_len = []
            queries_source0_len = []
            queries_target0_len = []
            for k in range( k_support ):
                ik2 = i * ( k_support + k_query ) + k
                while ik2 >= len( rr_s[j] ):
                    ik2 -= len( rr_s[j] )
                #print( "i,:",i, "j:", j, "len(rr[j]):", len(rr[j]), "ik2:", ik2 )
                pad_len = ss_max - rr_len_s[j][ik2]
                tmp_s = np.pad( rr_s[j][ik2], (0, pad_len), mode='constant', constant_values=(pad_idx_s, pad_idx_s ))
                supports_source0.append( tmp_s )
                pad_len = tt_max - rr_len_t[j][ik2]
                tmp_t = np.pad( rr_t[j][ik2], (0, pad_len), mode='constant', constant_values=(pad_idx_t, pad_idx_t ))
                supports_target0.append( tmp_t ) 
                supports_source0_len.append( rr_len_s[j][ik2])
                supports_target0_len.append( rr_len_t[j][ik2] ) 
            for k in range( k_query ):
                ik2 = i * ( k_support + k_query ) + k_support + k
                while ik2 >= len( rr_s[j] ):
                    ik2 -= len( rr_s[j] )
                pad_len = ss_max - rr_len_s[j][ik2]
                tmp_s = np.pad( rr_s[j][ik2], (0, pad_len), mode='constant', constant_values=(pad_idx_s, pad_idx_s ))
                queries_source0.append( tmp_s )
                pad_len = tt_max - rr_len_t[j][ik2]
                tmp_t = np.pad( rr_t[j][ik2], (0, pad_len), mode='constant', constant_values=(pad_idx_t, pad_idx_t ))
                queries_target0.append( tmp_t ) 
                queries_source0_len.append(rr_len_s[j][ik2])
                queries_target0_len.append( rr_len_t[j][ik2] ) 
            supports_source.append(supports_source0)
            supports_target.append( supports_target0 ) 
            queries_source.append(queries_source0)
            queries_target.append( queries_target0 ) 
            supports_source_len.append(supports_source0_len)
            supports_target_len.append( supports_target0_len ) 
            queries_source_len.append(queries_source0_len)
            queries_target_len.append( queries_target0_len ) 
        output0.append( supports_source )
        output0.append( supports_source_len )
        output0.append( supports_target )
        output0.append( supports_target_len )
        output0.append( queries_source )
        output0.append( queries_source_len )
        output0.append( queries_target )
        output0.append( queries_target_len )
        output.append( output0 )

    return output


In [5]:
# データセットを作る
with open('data_train.pkl', 'rb') as f:
    trainset = pickle.load(f)
#trainset = trainset[:2000]
#print( trainset[:10])
with open('data_val.pkl', 'rb') as f:
    valset = pickle.load(f)

pad_idx_s = idx_list['<pad>']
pad_idx_t = idx_list_en['<pad>']
    
num_ob_val = 10
ob_val = build_batch_set( valset, pad_idx_s, pad_idx_t, num_task = 3, k_support = 5, k_query = 5 )
ob_val = ob_val[:num_ob_val]
db_val = create_batch_of_tasks(ob_val, is_shuffle = False, outer_batch_size = len( ob_val ))

ob_train = build_batch_set( trainset, pad_idx_s, pad_idx_t, num_task = 3, k_support = 5, k_query = 5 )

del trainset, valset
gc.collect()

ss_max: 106
tt_max: 114
i: 0  count:  489
i: 1  count:  507
i: 2  count:  504
batch_max: 50
batch_min: 48
480
489
489
i: 0
ss_max: 306
tt_max: 332
i: 0  count:  998984
i: 1  count:  998999
i: 2  count:  999020
batch_max: 99902
batch_min: 99898
998980
998984
998984
i: 0
i: 10000
i: 20000
i: 30000
i: 40000
i: 50000
i: 60000
i: 70000
i: 80000
i: 90000


0

In [6]:
# dropout のにない PositionalEncoding
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        #return self.dropout(x)
        return x

In [7]:
# PositionalEmbedding
class PositionalEmbedding(nn.Module):
    '''
    位置埋め込み （Positional embedding）
    dim_embedding: 埋込み次元
    max_len      : 入力の最大系列長
    '''
    def __init__(self, dim_embedding: int, max_len: int=5000):
        super().__init__()

        self.pos_emb = nn.Embedding(max_len, dim_embedding)

    '''
    位置エンコーディングの順伝播
    x: 位置エンコーディングを埋め込む対象のテンソル,
       [バッチサイズ, 系列長, 埋め込み次元]
    '''
    def forward(self, x: torch.Tensor):
        seq = x.shape[1]
        positions = torch.arange(start=0, end=seq, step=1, device=x.device).to(torch.long)
        positions = self.pos_emb(positions)[:seq,:]
        
        return positions

In [8]:
#PositionalEmbedding の adaptation 用関数。
class fPositionalEmbedding(nn.Module):
    '''
    位置埋め込み （Positional embedding）
    dim_embedding: 埋込み次元
    max_len      : 入力の最大系列長
    '''
    def __init__(self, dim_embedding: int, name, max_len: int=5000):
        super().__init__()

        #self.pos_emb = nn.Embedding(max_len, dim_embedding)
        self.name = name
        
    '''
    位置エンコーディングの順伝播
    x: 位置エンコーディングを埋め込む対象のテンソル,
       [バッチサイズ, 系列長, 埋め込み次元]
    '''
    def forward(self, x: torch.Tensor, weights):
        seq = x.shape[1]
        positions = torch.arange(start=0, end=seq, step=1, device=x.device).to(torch.long)
        #positions = self.pos_emb(positions)[:seq,:]
        positions = F.embedding( positions, weights[self.name] )[:seq,:]
        
        return positions

In [9]:
# MultiHeadAttention 関数
class MHA(nn.Module):
    '''
    自己アテンション
    dim_hidden: 入力特徴量の次元
    num_heads : マルチヘッドアテンションのヘッド数
    qkv_bias  : クエリなどを生成する全結合層のバイアスの有無
    '''
    def __init__(self, dim_hidden: int, num_heads: int,
                 qkv_bias: bool=False):
        super().__init__()

        # 特徴量を各ヘッドのために分割するので、
        # 特徴量次元をヘッド数で割り切れるか検証
        assert dim_hidden % num_heads == 0

        self.num_heads = num_heads

        # ヘッド毎の特徴量次元
        dim_head = dim_hidden // num_heads

        # ソフトマックスのスケール値
        self.scale = dim_head ** -0.5

        # ヘッド毎にクエリ、キーおよびバリューを生成するための全結合層
        self.proj_in_q = nn.Linear(
            dim_hidden, dim_hidden, bias=qkv_bias)
        self.proj_in_k = nn.Linear(
            dim_hidden, dim_hidden, bias=qkv_bias)
        self.proj_in_v = nn.Linear(
            dim_hidden, dim_hidden, bias=qkv_bias)

        #self.in_proj_weight = nn.Parameter( torch.randn( dim_hidden * 3, dim_hidden ) )
        #self.in_proj_bias = nn.Parameter( torch.randn( dim_hidden * 3 ) )
        #self.in_proj_weight = Parameter( torch.empty( dim_hidden * 3, dim_hidden ) )
        #self.in_proj_bias = Parameter( torch.empty( dim_hidden * 3 ) )

        # 各ヘッドから得られた特徴量を一つにまとめる全結合層
        self.proj_out = nn.Linear(dim_hidden, dim_hidden)
        
    def split_head(self, x):
        x = torch.tensor_split(x, self.num_heads, dim = 2)
        x = torch.stack(x, dim = 1)
        return x

    '''
    順伝播関数
    x: 入力特徴量, [バッチサイズ, 特徴量数, 特徴量次元]
    '''
    def forward(self, q: torch.Tensor, k: torch.Tensor, v: torch.Tensor, mask: torch.Tensor):

        #bs_q, ns_q, ds_q = q.size()
        #bs_k, ns_k, ds_k = k.size()
        
        ##  k = v assumpotion
        #if q is k:
        #    qkv = q @ self.in_proj_weight.transpose(-2,-1) + self.in_proj_bias
        #    qkv = qkv.view( bs_q, ns_q, 3, ds_q )
        #    q, k, v = torch.unbind( qkv, dim = 2 )
        #else:
        #    W_q, W_kv = self.in_proj_weight.split([ds_q, ds_q * 2])
        #    b_q, b_kv = self.in_proj_bias.split([ds_q, ds_q * 2])
        #    q = q @ W_q.transpose(-2,-1) + b_q
        #    kv =  k @ W_kv.transpose(-2,-1) + b_kv
        #    kv = kv.view( bs_k, ns_k, 2, ds_k )
        #    k, v = torch.unbind( kv, dim = 2 )
        
        q = self.proj_in_q(q)
        k = self.proj_in_k(k)
        v = self.proj_in_v(v)
        
        q = self.split_head(q)
        k = self.split_head(k)
        v = self.split_head(v)

        # クエリとキーの行列積とアテンションの計算(今回マスクは不使用)
        # attnは[バッチサイズ, ヘッド数, 特徴量数, 特徴量数]
        attn = q.matmul(k.transpose(-2, -1))
        if mask is not None:
            attn = attn.masked_fill_(mask, -torch.finfo(torch.float).max)
        attn = (attn * self.scale).softmax(dim=-1)

        # アテンションとバリューの行列積によりバリューを収集
        # xは[バッチサイズ, ヘッド数, 特徴量数, ヘッドの特徴量次元]
        x = attn.matmul(v)

        # permute関数により
        # [バッチサイズ, 特徴量数, ヘッド数, ヘッドの特徴量次元]
        # flatten関数により全てのヘッドから得られる特徴量を連結して、
        # [バッチサイズ, 特徴量数, ヘッド数 * ヘッドの特徴量次元]
        x = x.permute(0, 2, 1, 3).flatten(2)
        x = self.proj_out(x)

        return x


In [10]:
# EncoderLayer 用 MultiHeadAttention 関数
class feMHA(nn.Module):
    '''
    自己アテンション
    dim_hidden: 入力特徴量の次元
    num_heads : マルチヘッドアテンションのヘッド数
    qkv_bias  : クエリなどを生成する全結合層のバイアスの有無
    '''
    def __init__(self, dim_hidden: int, num_heads: int,
                 qkv_bias: bool=False):
        super().__init__()

        # 特徴量を各ヘッドのために分割するので、
        # 特徴量次元をヘッド数で割り切れるか検証
        assert dim_hidden % num_heads == 0

        self.num_heads = num_heads

        # ヘッド毎の特徴量次元
        dim_head = dim_hidden // num_heads

        # ソフトマックスのスケール値
        self.scale = dim_head ** -0.5

        # ヘッド毎にクエリ、キーおよびバリューを生成するための全結合層
        #self.proj_in = nn.Linear(
        #    dim_hidden, dim_hidden * 3, bias=qkv_bias)

        # 各ヘッドから得られた特徴量を一つにまとめる全結合層
        #self.proj_out = nn.Linear(dim_hidden, dim_hidden)

    def split_head(self, x):
        x = torch.tensor_split(x, self.num_heads, dim = 2)
        x = torch.stack(x, dim = 1)
        return x

    '''
    順伝播関数
    x: 入力特徴量, [バッチサイズ, 特徴量数, 特徴量次元]
    '''
    def forward(self, q: torch.Tensor, k: torch.Tensor,  v: torch.Tensor, mask: torch.Tensor, i, weights ):
        bs, ns = q.shape[:2]

        #qkv = self.proj_in(x)
        q = F.linear(q, weights['encoder.encoder_layers.' + str(i) + '.attention.proj_in_q.weight'], weights['encoder.encoder_layers.' + str(i) + '.attention.proj_in_q.bias'])
        k = F.linear(k, weights['encoder.encoder_layers.' + str(i) + '.attention.proj_in_k.weight'], weights['encoder.encoder_layers.' + str(i) + '.attention.proj_in_k.bias'])
        v = F.linear(v, weights['encoder.encoder_layers.' + str(i) + '.attention.proj_in_v.weight'], weights['encoder.encoder_layers.' + str(i) + '.attention.proj_in_v.bias'])

        q = self.split_head( q )
        k = self.split_head( k )
        v = self.split_head( v )

        # クエリとキーの行列積とアテンションの計算(今回マスクは不使用)
        # attnは[バッチサイズ, ヘッド数, 特徴量数, 特徴量数]
        attn = q.matmul(k.transpose(-2, -1))
        if mask is not None:
            attn = attn.masked_fill_(mask, -torch.finfo(torch.float).max)
        attn = (attn * self.scale).softmax(dim=-1)

        # アテンションとバリューの行列積によりバリューを収集
        # xは[バッチサイズ, ヘッド数, 特徴量数, ヘッドの特徴量次元]
        x = attn.matmul(v)

        # permute関数により
        # [バッチサイズ, 特徴量数, ヘッド数, ヘッドの特徴量次元]
        # flatten関数により全てのヘッドから得られる特徴量を連結して、
        # [バッチサイズ, 特徴量数, ヘッド数 * ヘッドの特徴量次元]
        x = x.permute(0, 2, 1, 3).flatten(2)
        #x = self.proj_out(x)
        x = F.linear(x, weights['encoder.encoder_layers.' + str(i) + '.attention.proj_out.weight'], weights['encoder.encoder_layers.' + str(i) + '.attention.proj_out.bias'])

        return x

# DecoderLayer の selfattention 用、MHA 関数。
class fdsMHA(nn.Module):
    '''
    自己アテンション
    dim_hidden: 入力特徴量の次元
    num_heads : マルチヘッドアテンションのヘッド数
    qkv_bias  : クエリなどを生成する全結合層のバイアスの有無
    '''
    def __init__(self, dim_hidden: int, num_heads: int,
                 qkv_bias: bool=False):
        super().__init__()

        # 特徴量を各ヘッドのために分割するので、
        # 特徴量次元をヘッド数で割り切れるか検証
        assert dim_hidden % num_heads == 0

        self.num_heads = num_heads

        # ヘッド毎の特徴量次元
        dim_head = dim_hidden // num_heads

        # ソフトマックスのスケール値
        self.scale = dim_head ** -0.5

        # ヘッド毎にクエリ、キーおよびバリューを生成するための全結合層
        #self.proj_in = nn.Linear(
        #    dim_hidden, dim_hidden * 3, bias=qkv_bias)

        # 各ヘッドから得られた特徴量を一つにまとめる全結合層
        #self.proj_out = nn.Linear(dim_hidden, dim_hidden)

    def split_head(self, x):
        x = torch.tensor_split(x, self.num_heads, dim = 2)
        x = torch.stack(x, dim = 1)
        return x

    '''
    順伝播関数
    x: 入力特徴量, [バッチサイズ, 特徴量数, 特徴量次元]
    '''
    def forward(self, q: torch.Tensor, k: torch.Tensor,  v: torch.Tensor, mask: torch.Tensor, i, weights ):
        bs, ns = q.shape[:2]

        q = F.linear(q, weights['decoder.decoder_layers.' + str(i) + '.selfattn.proj_in_q.weight'], weights['decoder.decoder_layers.' + str(i) + '.selfattn.proj_in_q.bias'])
        k = F.linear(k, weights['decoder.decoder_layers.' + str(i) + '.selfattn.proj_in_k.weight'], weights['decoder.decoder_layers.' + str(i) + '.selfattn.proj_in_k.bias'])
        v = F.linear(v, weights['decoder.decoder_layers.' + str(i) + '.selfattn.proj_in_v.weight'], weights['decoder.decoder_layers.' + str(i) + '.selfattn.proj_in_v.bias'])

        q = self.split_head( q )
        k = self.split_head( k )
        v = self.split_head( v )

        # クエリとキーの行列積とアテンションの計算(今回マスクは不使用)
        # attnは[バッチサイズ, ヘッド数, 特徴量数, 特徴量数]
        attn = q.matmul(k.transpose(-2, -1))
        if mask is not None:
            attn = attn.masked_fill_(mask, -torch.finfo(torch.float).max)      
        attn = (attn * self.scale).softmax(dim=-1)

        # アテンションとバリューの行列積によりバリューを収集
        # xは[バッチサイズ, ヘッド数, 特徴量数, ヘッドの特徴量次元]
        x = attn.matmul(v)

        # permute関数により
        # [バッチサイズ, 特徴量数, ヘッド数, ヘッドの特徴量次元]
        # flatten関数により全てのヘッドから得られる特徴量を連結して、
        # [バッチサイズ, 特徴量数, ヘッド数 * ヘッドの特徴量次元]
        x = x.permute(0, 2, 1, 3).flatten(2)
        #x = self.proj_out(x)
        x = F.linear(x, weights['decoder.decoder_layers.' + str(i) + '.selfattn.proj_out.weight'], weights['decoder.decoder_layers.' + str(i) + '.selfattn.proj_out.bias'])

        return x

# DecoderLayer の crossattention 用、MHA 関数。
class fdcMHA(nn.Module):
    '''
    自己アテンション
    dim_hidden: 入力特徴量の次元
    num_heads : マルチヘッドアテンションのヘッド数
    qkv_bias  : クエリなどを生成する全結合層のバイアスの有無
    '''
    def __init__(self, dim_hidden: int, num_heads: int,
                 qkv_bias: bool=False):
        super().__init__()

        # 特徴量を各ヘッドのために分割するので、
        # 特徴量次元をヘッド数で割り切れるか検証
        assert dim_hidden % num_heads == 0

        self.num_heads = num_heads

        # ヘッド毎の特徴量次元
        dim_head = dim_hidden // num_heads

        # ソフトマックスのスケール値
        self.scale = dim_head ** -0.5

        # ヘッド毎にクエリ、キーおよびバリューを生成するための全結合層
        #self.proj_in = nn.Linear(
        #    dim_hidden, dim_hidden * 3, bias=qkv_bias)

        # 各ヘッドから得られた特徴量を一つにまとめる全結合層
        #self.proj_out = nn.Linear(dim_hidden, dim_hidden)

    def split_head(self, x):
        x = torch.tensor_split(x, self.num_heads, dim = 2)
        x = torch.stack(x, dim = 1)
        return x

    '''
    順伝播関数
    x: 入力特徴量, [バッチサイズ, 特徴量数, 特徴量次元]
    '''
    def forward(self, q: torch.Tensor, k: torch.Tensor,  v: torch.Tensor, mask: torch.Tensor, i, weights ):
        bs, ns = q.shape[:2]

        q = F.linear(q, weights['decoder.decoder_layers.' + str(i) + '.crossattn.proj_in_q.weight'], weights['decoder.decoder_layers.' + str(i) + '.crossattn.proj_in_q.bias'])
        k = F.linear(k, weights['decoder.decoder_layers.' + str(i) + '.crossattn.proj_in_k.weight'], weights['decoder.decoder_layers.' + str(i) + '.crossattn.proj_in_k.bias'])
        v = F.linear(v, weights['decoder.decoder_layers.' + str(i) + '.crossattn.proj_in_v.weight'], weights['decoder.decoder_layers.' + str(i) + '.crossattn.proj_in_v.bias'])

        q = self.split_head( q )
        k = self.split_head( k )
        v = self.split_head( v )

        # クエリとキーの行列積とアテンションの計算(今回マスクは不使用)
        # attnは[バッチサイズ, ヘッド数, 特徴量数, 特徴量数]
        attn = q.matmul(k.transpose(-2, -1))
        if mask is not None:
            attn = attn.masked_fill_(mask, -torch.finfo(torch.float).max)      
        attn = (attn * self.scale).softmax(dim=-1)

        # アテンションとバリューの行列積によりバリューを収集
        # xは[バッチサイズ, ヘッド数, 特徴量数, ヘッドの特徴量次元]
        x = attn.matmul(v)

        # permute関数により
        # [バッチサイズ, 特徴量数, ヘッド数, ヘッドの特徴量次元]
        # flatten関数により全てのヘッドから得られる特徴量を連結して、
        # [バッチサイズ, 特徴量数, ヘッド数 * ヘッドの特徴量次元]
        x = x.permute(0, 2, 1, 3).flatten(2)
        #x = self.proj_out(x)
        x = F.linear(x, weights['decoder.decoder_layers.' + str(i) + '.crossattn.proj_out.weight'], weights['decoder.decoder_layers.' + str(i) + '.crossattn.proj_out.bias'])

        return x


In [11]:
#FNN 関数
class FNN(nn.Module):
    '''
    Transformerエンコーダ内の順伝播型ニューラルネットワーク
    dim_hidden     : 入力特徴量の次元
    dim_feedforward: 中間特徴量の次元
    '''
    def __init__(self, dim_hidden: int, dim_feedforward: int):
        super().__init__()

        self.linear1 = nn.Linear(dim_hidden, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, dim_hidden)
        self.activation = nn.GELU()

    '''
    順伝播関数
    x: 入力特徴量, [バッチサイズ, 特徴量数, 特徴量次元]
    '''
    def forward(self, x: torch.Tensor):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)

        return x


In [12]:
# EncoderLayer 用、FNN adaptation 関数
class feFNN(nn.Module):
    '''
    Transformerエンコーダ内の順伝播型ニューラルネットワーク
    dim_hidden     : 入力特徴量の次元
    dim_feedforward: 中間特徴量の次元
    '''
    def __init__(self, dim_hidden: int, dim_feedforward: int):
        super().__init__()

        #self.linear1 = nn.Linear(dim_hidden, dim_feedforward)
        #self.linear2 = nn.Linear(dim_feedforward, dim_hidden)
        self.activation = nn.GELU()

    '''
    順伝播関数
    x: 入力特徴量, [バッチサイズ, 特徴量数, 特徴量次元]
    '''
    def forward(self, x: torch.Tensor, i, weights ):
        #x = self.linear1(x)
        x = F.linear(x, weights['encoder.encoder_layers.' + str(i) + '.fnn.linear1.weight'], weights['encoder.encoder_layers.' + str(i) + '.fnn.linear1.bias'])
        x = self.activation(x)
        #x = self.linear2(x)
        x = F.linear(x, weights['encoder.encoder_layers.' + str(i) + '.fnn.linear2.weight'], weights['encoder.encoder_layers.' + str(i) + '.fnn.linear2.bias'])

        return x


In [13]:
# DecoderLayer 用、FNN adaptation 関数
class fdFNN(nn.Module):
    '''
    Transformerエンコーダ内の順伝播型ニューラルネットワーク
    dim_hidden     : 入力特徴量の次元
    dim_feedforward: 中間特徴量の次元
    '''
    def __init__(self, dim_hidden: int, dim_feedforward: int):
        super().__init__()

        #self.linear1 = nn.Linear(dim_hidden, dim_feedforward)
        #self.linear2 = nn.Linear(dim_feedforward, dim_hidden)
        self.activation = nn.GELU()

    '''
    順伝播関数
    x: 入力特徴量, [バッチサイズ, 特徴量数, 特徴量次元]
    '''
    def forward(self, x: torch.Tensor, i, weights ):
        #x = self.linear1(x)
        x = F.linear(x, weights['decoder.decoder_layers.' + str(i) + '.fnn.linear1.weight'], weights['decoder.decoder_layers.' + str(i) + '.fnn.linear1.bias'])
        x = self.activation(x)
        #x = self.linear2(x)
        x = F.linear(x, weights['decoder.decoder_layers.' + str(i) + '.fnn.linear2.weight'], weights['decoder.decoder_layers.' + str(i) + '.fnn.linear2.bias'])

        return x


In [14]:
class TransformerEncoderLayer(nn.Module):
    '''
    Transformerエンコーダ層
    dim_hidden     : 入力特徴量の次元
    num_heads      : ヘッド数
    dim_feedforward: 中間特徴量の次元
    '''
    def __init__(self, dim_hidden: int, num_heads: int,
                 dim_feedforward: int, dropout: float = 0.0):
        super().__init__()

        self.attention = MHA(dim_hidden, num_heads, qkv_bias = True)
        self.fnn = FNN(dim_hidden, dim_feedforward)

        self.norm1 = nn.LayerNorm(dim_hidden)
        self.norm2 = nn.LayerNorm(dim_hidden)
        
        self.dropout = nn.Dropout( dropout )

    '''
    順伝播関数
    x: 入力特徴量, [バッチサイズ, 特徴量数, 特徴量次元]
    '''
    def forward(self, x: torch.Tensor, mask: torch.Tensor):
        ''' B2T
        x0 = x
        x = self.attention( x, x, x, mask )
        x = self.dropout( x )
        x = x + x0
        x = self.norm1(x)
        x1 = x
        x = self.fnn( x )
        x = self.dropout( x )
        x =  x + x1 + x0
        x = self.norm2( x )
        '''
        #pre-LN
        x0 = x
        x = self.norm1(x)
        x = self.attention( x, x, x, mask )
        x = self.dropout( x )
        x = x + x0
        x1 = x
        x = self.norm2( x )
        x = self.fnn( x )
        x = self.dropout( x )
        x =  x + x1
        
        return x 


In [15]:
class fTransformerEncoderLayer(nn.Module):
    '''
    Transformerエンコーダ層
    dim_hidden     : 入力特徴量の次元
    num_heads      : ヘッド数
    dim_feedforward: 中間特徴量の次元
    '''
    def __init__(self, dim_hidden: int, num_heads: int,
                 dim_feedforward: int, dropout: float = 0.0):
        super().__init__()

        self.fattention = feMHA(dim_hidden, num_heads)
        self.fffn = feFNN(dim_hidden, dim_feedforward)
        self.dim_hidden = dim_hidden
        self.dropout = nn.Dropout( dropout )
        
    '''
    順伝播関数
    x: 入力特徴量, [バッチサイズ, 特徴量数, 特徴量次元]
    '''
    def forward(self, x: torch.Tensor, mask: torch.Tensor, i, weights ):
        '''B2T
        x0 = x
        x = self.fattention( x, x, x, mask, i, weights )
        x = self.dropout( x )
        x = x + x0
        x = F.layer_norm(x, (self.dim_hidden,), weight=weights['encoder.encoder_layers.' + str(i) + '.norm1.weight'], bias=weights['encoder.encoder_layers.' + str(i) + '.norm1.bias'], eps=1e-05)
        x1 = x
        x = self.fffn( x, i, weights ) 
        x = self.dropout( x )
        x = x + x1 + x0
        x = F.layer_norm(x, (self.dim_hidden,), weight=weights['encoder.encoder_layers.' + str(i) + '.norm2.weight'], bias=weights['encoder.encoder_layers.' + str(i) + '.norm2.bias'], eps=1e-05)
        '''
        #pre-LN
        x0 = x
        x = F.layer_norm(x, (self.dim_hidden,), weight=weights['encoder.encoder_layers.' + str(i) + '.norm1.weight'], bias=weights['encoder.encoder_layers.' + str(i) + '.norm1.bias'], eps=1e-05)
        x = self.fattention( x, x, x, mask, i, weights )
        x = self.dropout( x )
        x = x + x0
        x1 = x
        x = F.layer_norm(x, (self.dim_hidden,), weight=weights['encoder.encoder_layers.' + str(i) + '.norm2.weight'], bias=weights['encoder.encoder_layers.' + str(i) + '.norm2.bias'], eps=1e-05)
        x = self.fffn( x, i, weights ) 
        x = self.dropout( x )
        x = x + x1

        return x

In [16]:
class TransformerDecoderLayer(nn.Module):
    '''
    Transformerエンコーダ層
    dim_hidden     : 入力特徴量の次元
    num_heads      : ヘッド数
    dim_feedforward: 中間特徴量の次元
    '''
    def __init__(self, dim_hidden: int, num_heads: int,
                 dim_feedforward: int, dropout: float=0.0):
        super().__init__()

        self.selfattn = MHA(dim_hidden, num_heads, qkv_bias = True)
        self.crossattn = MHA(dim_hidden, num_heads, qkv_bias = True)
        self.fnn = FNN(dim_hidden, dim_feedforward)

        self.norm1 = nn.LayerNorm(dim_hidden)
        self.norm2 = nn.LayerNorm(dim_hidden)
        self.norm3 = nn.LayerNorm(dim_hidden)
        
        self.dropout = nn.Dropout(dropout)

    '''
    順伝播関数
    x: 入力特徴量, [バッチサイズ, 特徴量数, 特徴量次元]
    '''
    def forward(self, x: torch.Tensor, y: torch.Tensor,  self_mask: torch.Tensor,  cross_mask: torch.Tensor):
        '''B2T
        x0 = x
        x = self.selfattn( x, x, x, self_mask )
        x = self.dropout( x )
        x = x + x0
        x = self.norm1( x )
        x1 = x
        x = self.crossattn( x, y, y, cross_mask )
        x = self.dropout( x )
        x = x + x1
        x = self.norm2( x )
        x2 = x
        x = self.fnn( x )
        x = self.dropout( x )
        x = x + x2 + x0
        x = self.norm3( x )
        '''
        #pre-LN
        x0 = x
        x = self.norm1( x )
        x = self.selfattn( x, x, x, self_mask )
        x = self.dropout( x )
        x = x + x0
        x1 = x
        x = self.norm2( x )
        x = self.crossattn( x, y, y, cross_mask )
        x = self.dropout( x )
        x = x + x1
        x2 = x
        x = self.norm3( x )
        x = self.fnn( x )
        x = self.dropout( x )
        x = x + x2
        
        return x 


In [17]:
class fTransformerDecoderLayer(nn.Module):
    '''
    Transformerエンコーダ層
    dim_hidden     : 入力特徴量の次元
    num_heads      : ヘッド数
    dim_feedforward: 中間特徴量の次元
    '''
    def __init__(self, dim_hidden: int, num_heads: int,
                 dim_feedforward: int, dropout: float = 0.0):
        super().__init__()

        self.fselfattn = fdsMHA(dim_hidden, num_heads)
        self.fcrossattn = fdcMHA(dim_hidden, num_heads)
        self.fffn = fdFNN(dim_hidden, dim_feedforward)
        self.dim_hidden = dim_hidden
        self.dropout = nn.Dropout(dropout)

        #self.norm1 = nn.LayerNorm(dim_hidden)
        #self.norm2 = nn.LayerNorm(dim_hidden)
        #self.norm3 = nn.LayerNorm(dim_hidden)

    '''
    順伝播関数
    x: 入力特徴量, [バッチサイズ, 特徴量数, 特徴量次元]
    '''
    def forward(self, x: torch.Tensor, y: torch.Tensor,  self_mask: torch.Tensor,  cross_mask: torch.Tensor, i, weights):
        '''B2T
        x0 = x
        x = self.fselfattn( x, x, x, self_mask, i, weights )
        x = self.dropout( x )
        x = x + x0
        x = F.layer_norm(x, (self.dim_hidden,), weight=weights['decoder.decoder_layers.' + str(i) + '.norm1.weight'], bias=weights['decoder.decoder_layers.' + str(i) + '.norm1.bias'], eps=1e-05)
        x1 = x
        x = self.fcrossattn( x, y, y, cross_mask, i, weights )
        x = self.dropout( x )
        x = x + x1
        x = F.layer_norm(x, (self.dim_hidden,), weight=weights['decoder.decoder_layers.' + str(i) + '.norm2.weight'], bias=weights['decoder.decoder_layers.' + str(i) + '.norm2.bias'], eps=1e-05)
        x2 = x
        x = self.fffn( x, i, weights )
        x = self.dropout( x ) 
        x = x + x2 + x0
        x = F.layer_norm(x, (self.dim_hidden,), weight=weights['decoder.decoder_layers.' + str(i) + '.norm3.weight'], bias=weights['decoder.decoder_layers.' + str(i) + '.norm3.bias'], eps=1e-05)
        '''
        #pre-LN
        x0 = x
        x = F.layer_norm(x, (self.dim_hidden,), weight=weights['decoder.decoder_layers.' + str(i) + '.norm1.weight'], bias=weights['decoder.decoder_layers.' + str(i) + '.norm1.bias'], eps=1e-05)
        x = self.fselfattn( x, x, x, self_mask, i, weights )
        x = self.dropout( x )
        x = x + x0
        x1 = x
        x = F.layer_norm(x, (self.dim_hidden,), weight=weights['decoder.decoder_layers.' + str(i) + '.norm2.weight'], bias=weights['decoder.decoder_layers.' + str(i) + '.norm2.bias'], eps=1e-05)
        x = self.fcrossattn( x, y, y, cross_mask, i, weights )
        x = self.dropout( x )
        x = x + x1
        x2 = x
        x = F.layer_norm(x, (self.dim_hidden,), weight=weights['decoder.decoder_layers.' + str(i) + '.norm3.weight'], bias=weights['decoder.decoder_layers.' + str(i) + '.norm3.bias'], eps=1e-05)
        x = self.fffn( x, i, weights )
        x = self.dropout( x ) 
        x = x + x2
        
        return x 


In [18]:
class TransformerEncoder(nn.Module):
    '''
    dim_hidden     : 入力特徴量の次元
    dim_feedforward: FNNにおける中間特徴量の次元
    num_heads      : マルチヘッドアテンションのヘッド数
    num_layers     : Transformerエンコーダの層数
    '''
    def __init__(self, text_vocab_size: int, dim_embedding: int, dim_feedforward: int,
                 num_heads: int,  num_layers: int, pad_index:int, dropout: float = 0.1 ):
        super().__init__()

        # 単語埋め込み
        self.embed = nn.Embedding(
            text_vocab_size, dim_embedding, padding_idx=pad_index)        
        
        # 位置エンコーディング
        #self.pos_enc = PositionalEncoding(dim_embedding)
        self.pos_emb = PositionalEmbedding(dim_embedding)
        #self.dropout = nn.Dropout( dropout )
        
        # Transformerエンコーダ層
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(dim_hidden=dim_embedding, num_heads=num_heads, dim_feedforward = dim_feedforward, dropout = dropout )
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(dim_embedding)
       
        self.pad_index = pad_index
        self.dim_embedding = dim_embedding
        self.num_heads = num_heads

    '''
    順伝播関数
    x           : 入力, [バッチサイズ, 入力チャネル数, 高さ, 幅]
    return_embed: 特徴量を返すかロジットを返すかを選択する真偽値
    '''
    def forward(self, src: torch.Tensor, src_mask: torch.Tensor=None, src_padding_mask: torch.Tensor=None ):

        if src_padding_mask is not None and src_mask is None:
            mask = src_padding_mask[:,None,None,:]
            mask = mask.expand( (-1, self.num_heads, src.size(1), -1) )
        elif src_padding_mask is not None and src_mask is not None:
            mask1 = src_padding_mask[:,None,None,:]
            mask1 = mask1.expand( (-1, self.num_heads, src.size(1), -1 ) )
            mask2 = src_mask[None,None,:,:]
            mask2 = mask2.expand( ( src.size(0), self.num_heads, -1, -1 ) )
            mask = torch.logical_or(mask1, mask2 )
        elif src_padding_mask is None and src_mask is not None:
            mask = src_mask[None,None,:,:]
            mask = mask.padding( ( src.size(0), src.size(1), -1, -1 ) )
        else:
            mask = None

        x = self.embed( src ) * math.sqrt( self.dim_embedding )

        #x = self.pos_enc( x )
        position = self.pos_emb( x )
        x = x + position
        #x = self.dropout( x )
        #x = self.norm( x )
        
        # Transformerエンコーダ層を適用
        for layer in self.encoder_layers:
            x = layer( x, mask )

        x = self.norm(x)

        return x


In [19]:
class fTransformerEncoder(nn.Module):
    '''
    dim_hidden     : 入力特徴量の次元
    dim_feedforward: FNNにおける中間特徴量の次元
    num_heads      : マルチヘッドアテンションのヘッド数
    num_layers     : Transformerエンコーダの層数
    '''
    def __init__(self, text_vocab_size: int, dim_embedding: int, dim_feedforward: int,
                 num_heads: int,  num_layers: int, pad_idx:int, dropout: float = 0.1 ):
        super().__init__()

        # 単語埋め込み
        #self.embed = nn.Embedding(
        #    text_vocab_size, dim_embedding, padding_idx=pad_index)        
        
        # 位置エンコーディング
        #self.pos_enc = PositionalEncoding(dim_embedding)
        self.fpos_emb = fPositionalEmbedding(dim_embedding, "encoder.pos_emb.pos_emb.weight" )
        #self.dropout = nn.Dropout( dropout )
        
        # Transformerエンコーダ層
        self.fenclayer = fTransformerEncoderLayer(dim_hidden=dim_embedding, num_heads=num_heads, dim_feedforward = dim_feedforward, dropout = dropout )
        
        # ロジットを生成する前のレイヤー正規化と全結合
        #self.norm = nn.LayerNorm(dim_embedding)
        
        self.pad_idx = pad_idx
        self.dim_embedding = dim_embedding
        self.num_heads = num_heads
        self.num_layers = num_layers

    '''
    順伝播関数
    x           : 入力, [バッチサイズ, 入力チャネル数, 高さ, 幅]
    return_embed: 特徴量を返すかロジットを返すかを選択する真偽値
    '''
    def forward(self, src: torch.Tensor, src_mask: torch.Tensor=None, src_padding_mask: torch.Tensor=None, weights = None ):

        if src_padding_mask is not None and src_mask is None:
            mask = src_padding_mask[:,None,None,:]
            mask = mask.expand( (-1, self.num_heads, src.size(1), -1) )
        elif src_padding_mask is not None and src_mask is not None:
            mask1 = src_padding_mask[:,None,None,:]
            mask1 = mask1.expand( (-1, self.num_heads, src.size(1), -1 ) )
            mask2 = src_mask[None,None,:,:]
            mask2 = mask2.expand( ( src.size(0), self.num_heads, -1, -1 ) )
            mask = torch.logical_or(mask1, mask2 )
        elif src_padding_mask is None and src_mask is not None:
            mask = src_mask[None,None,:,:]
            mask = mask.padding( ( src.size(0), src.size(1), -1, -1 ) )
        else:
            mask = None

        #x = self.embed( src ) * math.sqrt( self.dim_embedding )
        x = F.embedding( src, weights['encoder.embed.weight'], padding_idx = self.pad_idx )  * math.sqrt( self.dim_embedding )
        
        #x = self.pos_enc( x )
        position = self.fpos_emb( x, weights )
        x = x + position
        #x = self.dropout( x )
        ##x = self.norm(x)
        #x = F.layer_norm(x, (self.dim_embedding,), weight=weights['encoder.norm.weight'], bias=weights['encoder.norm.bias'], eps=1e-05)
    
        # Transformerエンコーダ層を適用
        #for layer in self.encoder_layers:
        #    x = layer( x, mask )
        for block in range( self.num_layers ):
            x = self.fenclayer(x, mask, block, weights )

        #x = self.norm(x)
        x = F.layer_norm(x, (self.dim_embedding,), weight=weights['encoder.norm.weight'], bias=weights['encoder.norm.bias'], eps=1e-05)
        
        return x    

In [20]:
class TransformerDecoder(nn.Module):
    '''
    CaptioningTransformerのコンストラクタ
    dim_embedding  : 埋め込み次元
    dim_feedforward: FNNの中間特徴次元
    num_heads      : マルチヘッドアテンションのヘッド数
    num_layers     : Transformerデコーダ層の数
    vocab_size     : 辞書の次元
    pad_index     : NULLのID
    dropout        : ドロップアウト確率
    '''
    def __init__(self, vocab_size: int, dim_embedding: int, dim_feedforward: int,
                 num_heads: int, num_layers: int, 
                 pad_index: int, dropout: float=0.1, ds_rate: float=0.1 ):
        super().__init__()

        # 単語埋め込み
        self.embed = nn.Embedding(
            vocab_size, dim_embedding, padding_idx=pad_index)
        
        # 位置エンコーディング
        #self.pos_enc = PositionalEncoding(dim_embedding)
        self.pos_emb = PositionalEmbedding(dim_embedding)
        #self.dropout = nn.Dropout( dropout )

        # Transformerデコーダ
        self.decoder_layers = nn.ModuleList([
            TransformerDecoderLayer(dim_hidden=dim_embedding, num_heads=num_heads, dim_feedforward = dim_feedforward, dropout = dropout)
            for _ in range(num_layers)
        ])

        # ロジットを生成する前のレイヤー正規化と全結合
        #self.norm = nn.LayerNorm(dim_embedding)
        
        # 単語出力分布計算
        self.linear = nn.Linear(dim_embedding, vocab_size)
        
        self.pad_index = pad_index
        self.dim_embedding = dim_embedding
        self.num_heads = num_heads

    ''' CaptioningTransformerの順伝播処理
    features: 画像特徴量 [バッチサイズ, 埋め込み次元]
    captions: 正解キャプション [バッチサイズ, 系列長]

    '''

    def forward(self, features: torch.Tensor, captions: torch.Tensor, memory_padding_mask: torch.Tensor=None, \
                tgt_padding_mask: torch.Tensor=None, tgt_mask: torch.Tensor=None ):

        tgt = captions
        if tgt_padding_mask is not None and tgt_mask is not None:
            self_mask1 = tgt_padding_mask[:,None,None,:]
            self_mask1 = self_mask1.expand( (-1, self.num_heads, tgt.size(1), -1) )
            self_mask2 = tgt_mask[None,None,:,:]
            self_mask2 = self_mask2.expand( (tgt.size(0), self.num_heads, -1, -1 ))
            self_mask = torch.logical_or(self_mask1, self_mask2 )
        elif tgt_padding_mask is not None and tgt_mask is None:
            self_mask = tgt_padding_mask[:,None,None,:]
            self_mask = self_mask.expand( (-1, self.num_heads, tgt.size(1), -1 ) )
        elif tgt_padding_mask is None and tgt_mask is not None:
            self_mask = tgt_mask[None,None,:,:]
            self_mask = self_mask.expand( (tgt.size(0), self.num_heads, -1, -1 ))
        elif tgt_padding_mask is None and tgt_mask is None:
            self_mask = None
            
        if memory_padding_mask is not None:
            cross_mask = memory_padding_mask[:,None,None,:]
            cross_mask = cross_mask.expand((-1,self.num_heads, tgt.size(1), -1))
        else:
            cross_mask = None
        
        # 単語埋め込み [バッチサイズ, 系列長]
        # -> [バッチサイズ, 系列長, 埋め込み次元]
        embeddings = self.embed(captions) * math.sqrt( self.dim_embedding )
        seq = embeddings.shape[1]
        
        # 位置エンコーディング
        #embeddings = self.pos_enc( embeddings )
        positions = self.pos_emb(embeddings)
        embeddings = embeddings + positions
        #embeddings = self.dropout( embeddings )
        #embeddings = self.norm(embeddings)
        
        # Transformerデコーダでキャプション生成
        # 画像の特徴も入力する
        for layer in self.decoder_layers:
            embeddings = layer( embeddings, features, self_mask, cross_mask )
        
        # [バッチサイズ, 系列長, 埋め込み次元]
        # -> [バッチサイズ, 系列長, 辞書の次元]
        preds = self.linear(embeddings)
        return preds

In [21]:
class fTransformerDecoder(nn.Module):
    '''
    CaptioningTransformerのコンストラクタ
    dim_embedding  : 埋め込み次元
    dim_feedforward: FNNの中間特徴次元
    num_heads      : マルチヘッドアテンションのヘッド数
    num_layers     : Transformerデコーダ層の数
    vocab_size     : 辞書の次元
    pad_index     : NULLのID
    dropout        : ドロップアウト確率
    '''
    def __init__(self, vocab_size: int, dim_embedding: int, dim_feedforward: int,
                 num_heads: int, num_layers: int, 
                 pad_idx: int, dropout: float=0.1, ds_rate: float=0.1 ):
        super().__init__()

        # 単語埋め込み
        #self.embed = nn.Embedding(
        #    vocab_size, dim_embedding, padding_idx=pad_index)
        
        # 位置エンコーディング
        #self.pos_enc = PositionalEncoding(dim_embedding)
        self.fpos_emb = fPositionalEmbedding(dim_embedding, "decoder.pos_emb.pos_emb.weight" )
        #self.dropout = nn.Dropout( dropout )
        
        # Transformerデコーダ
        self.fdeclayer = fTransformerDecoderLayer(dim_hidden=dim_embedding, num_heads=num_heads, dim_feedforward = dim_feedforward, dropout = dropout )
        
        # 単語出力分布計算
        #self.linear = nn.Linear(dim_embedding, vocab_size)
        
        self.pad_idx = pad_idx
        self.dim_embedding = dim_embedding
        self.num_heads = num_heads
        self.num_layers = num_layers

    ''' CaptioningTransformerの順伝播処理
    features: 画像特徴量 [バッチサイズ, 埋め込み次元]
    captions: 正解キャプション [バッチサイズ, 系列長]

    '''
    def forward(self, features: torch.Tensor, captions: torch.Tensor, memory_padding_mask: torch.Tensor=None, \
                tgt_padding_mask: torch.Tensor=None, tgt_mask: torch.Tensor=None, weights = None ):

        tgt = captions
        if tgt_padding_mask is not None and tgt_mask is not None:
            self_mask1 = tgt_padding_mask[:,None,None,:]
            self_mask1 = self_mask1.expand( (-1, self.num_heads, tgt.size(1), -1) )
            self_mask2 = tgt_mask[None,None,:,:]
            self_mask2 = self_mask2.expand( (tgt.size(0), self.num_heads, -1, -1 ))
            self_mask = torch.logical_or(self_mask1, self_mask2 )
        elif tgt_padding_mask is not None and tgt_mask is None:
            self_mask = tgt_padding_mask[:,None,None,:]
            self_mask = self_mask.expand( (-1, self.num_heads, tgt.size(1), -1 ) )
        elif tgt_padding_mask is None and tgt_mask is not None:
            self_mask = tgt_mask[None,None,:,:]
            self_mask = self_mask.expand( (tgt.size(0), self.num_heads, -1, -1 ))
        elif tgt_padding_mask is None and tgt_mask is None:
            self_mask = None
            
        if memory_padding_mask is not None:
            cross_mask = memory_padding_mask[:,None,None,:]
            cross_mask = cross_mask.expand((-1,self.num_heads, tgt.size(1), -1))
        else:
            cross_mask = None
        
        # 単語埋め込み [バッチサイズ, 系列長]
        # -> [バッチサイズ, 系列長, 埋め込み次元]
        embeddings = F.embedding( captions, weights['decoder.embed.weight'], padding_idx = self.pad_idx )  * math.sqrt( self.dim_embedding )
        seq = embeddings.shape[1]
        
        # 位置エンコーディング
        #embeddings = self.pos_enc(embeddings)
        positions = self.fpos_emb( embeddings, weights )
        embeddings = embeddings + positions
        #embeddings = self.dropout( embeddings )
        ##embeddings = self.norm(embeddings)
        #embeddings = F.layer_norm(embeddings, (self.dim_embedding,), weight=weights['decoder.norm.weight'], bias=weights['decoder.norm.bias'], eps=1e-05)
        
        # Transformerデコーダでキャプション生成
        # 画像の特徴も入力する
        #for layer in self.decoder_layers:
        #    #embeddings = layer( embeddings, features, tgt_key_padding_mask = padding_mask_tgt, \
        #    #                                memory_key_padding_mask = padding_mask_src, tgt_is_causal = True, tgt_mask = mask_tgt )
        for block in range( self.num_layers ):
            embeddings = self.fdeclayer(embeddings, features, self_mask, cross_mask, block, weights )
  
        # [バッチサイズ, 系列長, 埋め込み次元]
        # -> [バッチサイズ, 系列長, 辞書の次元]
        #preds = self.linear(embeddings)
        preds = F.linear( embeddings, weights['decoder.linear.weight'], weights['decoder.linear.bias'] )

        return preds

In [22]:
class Transformer(nn.Module):
    def __init__(self, dim_embedding: int, dim_feedforward: int,
                 num_heads: int, num_layers: int, enc_vocab_size: int, dec_vocab_size: int,
                 j_pad_index: int,e_pad_index: int, dropout: float=0.1, ds_rate: float=0.1 ):
        super().__init__()

        self.encoder = TransformerEncoder(enc_vocab_size, dim_embedding, dim_feedforward, num_heads, num_layers, j_pad_index, dropout )
        self.decoder = TransformerDecoder(dec_vocab_size, dim_embedding, dim_feedforward, num_heads, num_layers, e_pad_index, dropout )
        self.fencoder = fTransformerEncoder(enc_vocab_size, dim_embedding, dim_feedforward, num_heads, num_layers, j_pad_index, dropout )
        self.fdecoder = fTransformerDecoder(dec_vocab_size, dim_embedding, dim_feedforward, num_heads, num_layers, e_pad_index, dropout )
        self.j_pad_index = j_pad_index
        self.e_pad_index = e_pad_index

        self._reset_parameters()
        #self._reset_parameters2()
    
    def _reset_parameters(self):
        r"""Initiate parameters in the transformer model."""
        for p in self.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)
                #xavier_normal_(p)
                #kaiming_uniform_(p)
                #kaiming_normal_(p)

    #def _reset_parameters2(self):
    #    for module in self.modules():
    #        if isinstance(module, nn.Linear):
    #            nn.init.normal_(module.weight, mean=0.0, std=0.02)
    #            if module.bias is not None:
    #                nn.init.zeros_(module.bias)
    #        elif isinstance(module, nn.Embedding):
    #            nn.init.normal_(module.weight, mean=0.0, std=0.02)
    #        elif isinstance(module, nn.LayerNorm):
    #            nn.init.zeros_(module.bias)
    #            nn.init.ones_(module.weight)
        
    ''' CaptioningTransformerの順伝播処理
    features: 画像特徴量 [バッチサイズ, 埋め込み次元]
    captions: 正解キャプション [バッチサイズ, 系列長]

    '''
    def forward(self, text, dec_input):

        seq_len_src = text.shape[1]
        seq_len_tgt = dec_input.shape[1]

        mask_tgt = nn.Transformer.generate_square_subsequent_mask( seq_len_tgt, dtype=bool ).to(device)
        mask_src = torch.zeros((seq_len_src, seq_len_src), device=device).type(torch.bool)

        padding_mask_src = (text == idx_list['<pad>'])
        padding_mask_tgt = (dec_input == idx_list_en['<pad>'])
    
        x = self.encoder( text, mask_src, padding_mask_src )
        preds = self.decoder(x,dec_input, padding_mask_src, padding_mask_tgt, mask_tgt )

        return preds
    
    def adaptation(self, text, dec_input, weights):

        seq_len_src = text.shape[1]
        seq_len_tgt = dec_input.shape[1]

        mask_tgt = nn.Transformer.generate_square_subsequent_mask( seq_len_tgt, dtype=bool ).to(device)
        mask_src = torch.zeros((seq_len_src, seq_len_src), device=device).type(torch.bool)

        padding_mask_src = (text == idx_list['<pad>']).to(text.device )
        padding_mask_tgt = (dec_input == idx_list_en['<pad>']).to(text.device )
        
        x = self.fencoder( text, mask_src, padding_mask_src, weights)
        preds = self.fdecoder( x, dec_input, padding_mask_src, padding_mask_tgt, mask_tgt, weights )
        
        return preds


In [23]:
class Adaptation(nn.Module):
    def __init__(self, model, loss_fn, outer_optimizer, device, id_to_word, derivative_change_rate, clip_max ):
        super().__init__()

        self.model = model
        self.loss_fn = loss_fn
        self.outer_optimizer = outer_optimizer
        self.device = device
        self.id_to_word = id_to_word
        self.derivative_change_rate = derivative_change_rate
    
    def forward(self, batch, train_step, train, lr1, display, progress_rate):

        x_train = torch.tensor( np.array( batch[0] ) ).to(self.device) #support tokeinzer.encode text データ
        x_len_train = torch.tensor( batch[1] ).to(self.device) #support attention mask データ
        y_train = torch.tensor( np.array( batch[2] ) ).to(self.device) #support ラベルデータ
        y_len_train = torch.tensor( batch[3] ).to(self.device)#support attention mask データ
        x_val = torch.tensor( np.array( batch[4] ) ).to(self.device)   #query  tokeinzer.encode text データ
        x_len_val = torch.tensor( batch[5] ).to(self.device)   #query attention mask データ
        y_val = torch.tensor( np.array( batch[6] )  ).to(self.device)  #query ラベルデータ
        y_len_val = torch.tensor( batch[7] ).to(self.device)   #query attention mask データ

        num_task = len( x_train )

        outer_loss_item = 0
        total_error = 0
        total_token_length = 0

        if train:
            weights0 = OrderedDict(self.model.named_parameters())#今回の基準パラメータ, 
        for idx in range(x_train.size(0)): # task
            if train:
                weights = weights0
            fast_weights = OrderedDict(self.model.named_parameters())#今回の基準パラメータ, 
       
            # batch 抽出
            input_x_len = x_len_train[idx]
            input_y_len = y_len_train[idx]        
            input_x = x_train[idx][:,:max(input_x_len)].to(self.device)
            input_y = y_train[idx][:,:max(input_y_len)].to(self.device).long()
            input_y1 = input_y[:,:-1]
            input_y2 = input_y[:,1:]

            if display:
                print('----Task',idx, '----')

            # タスクごとの損失の計算
            loss_item = 0
            for iter in range(train_step): # train_step のループ
                self.model.train()
                logits = self.model.adaptation(input_x, input_y1, fast_weights )
                loss0 = self.loss_fn(logits.transpose(1,2), input_y2  )
                loss_item += loss0.item()
                if progress_rate > self.derivative_change_rate:
                    gradients = torch.autograd.grad(loss0, fast_weights.values(), create_graph = True )
                else:
                    gradients = torch.autograd.grad(loss0, fast_weights.values() )
                fast_weights = OrderedDict((name, param - lr1 * grad) for ((name, param),  grad) in zip(fast_weights.items(), gradients))

                del logits, gradients, loss0
                torch.cuda.empty_cache()  

            loss_item = loss_item / train_step 

            if display:
                print("Inner Loss: ", loss_item)

            del loss_item, input_x, input_y, input_y1, input_y2, input_x_len, input_y_len
            torch.cuda.empty_cache()  
        
            # query データからバッチ抽出
            input_x_len = x_len_val[idx]
            input_y_len = y_len_val[idx]        
            input_x = x_val[idx][:,:max(input_x_len)].to(self.device)
            input_y = y_val[idx][:,:max(input_y_len)].to(self.device).long()
            input_y1 = input_y[:,:-1]
            input_y2 = input_y[:,1:]
        
            # 訓練時に query データ（inner_batch = 1, query_k * 2 クラス )　で二番目の損失関数の各タスクについての総和を求める。
            #if train:
            if train:
                self.model.train()
                logits = self.model.adaptation(input_x, input_y1, fast_weights )
            else:
                self.model.eval()
                with torch.no_grad():
                    logits = self.model.adaptation(input_x, input_y1, fast_weights )


            # 更新したモデルパラメーターを用いて、損失と精度を求める。

            pred_label_id = torch.argmax( logits, dim = 2 )
            #print( "pred_label_id[0]:", pred_label_id[0] )
            predict = []
            target = []
            for i0, (pred, orig) in enumerate( zip( pred_label_id, input_y2 )):
                hypo = []
                for m in pred:
                    hypo.append(self.id_to_word[m.item()])
                    if self.id_to_word[m.item()] == '<eos>':
                        break
                predict.append( hypo )
                reference = []
                for m in orig:
                    reference.append(self.id_to_word[m.item()])
                    if self.id_to_word[m.item()] == '<eos>':
                        break
                target.append( reference )    
                   # 認識誤りを計算
                (error, substitute, 
                    delete, insert, ref_length) = \
                       levenshtein.calculate_error(hypo,
                                               reference)
                    
                # 誤り文字数を累積する
                total_error += error
                # 文字の総数を累積する
                total_token_length += ref_length                
            if idx == 0:
                #print( "pred_label_id:", pred_label_id )
                for i4, (pred, tar) in enumerate( zip( predict, target ) ):
                    if i4 >= 2:
                        break
                    if display:
                        if train:
                            print( "train pred:", ' '.join(pred) )
                            print( "train targ:", ' '.join(tar)  )
                        else: 
                            print( "val pred:", ' '.join(pred) )
                            print( "val targ:", ' '.join(tar)  )

            outer_loss0 = self.loss_fn(logits.transpose(1,2), input_y2 )
            outer_loss_item += outer_loss0.item()
            if train:
                tmp = torch.autograd.grad( outer_loss0, weights.values() )
                if idx ==0:
                    gradients2 = list(tmp)
                else:
                    gradients2 = [x + y for x, y in zip(gradients2, list(tmp))]
                del tmp, weights
            del outer_loss0, logits, input_x, input_y,  input_y1, input_y2, input_x_len, input_y_len, fast_weights
            torch.cuda.empty_cache()  
        
        if train:
            self.outer_optimizer.zero_grad()
            for i, params in enumerate(self.model.parameters()):
                params.grad = gradients2[i]
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), clip_max)
            self.outer_optimizer.step()
            del gradients2
            torch.cuda.empty_cache()  
    
        avg_error = total_error / total_token_length
        outer_loss_item = outer_loss_item / x_train.size(0)
    
        return outer_loss_item, avg_error

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device( "cpu")
epoch_num = 3
dim_hidden = 768
dim_feedforward = dim_hidden * 4
heads = 12
layers =8
dropout = 0.1
derivative_change_rate = 0.01
clip_max = 1.0
#dim_hidden = 256
#dim_feedforward = dim_hidden * 4
#heads = 4
#layers =4
#dropout = 0.0
model = Transformer(dim_hidden, dim_feedforward, heads, layers, enc_vocab_size, dec_vocab_size,
                    idx_list['<pad>'], idx_list_en['<pad>'], dropout = dropout ).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=idx_list_en['<pad>'])
loss_fn = criterion
#optimizer = optim.Adam(model.parameters(), lr=0.00001)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
adaptation = Adaptation( model = model, loss_fn = loss_fn, outer_optimizer = optimizer, device = device,
                        id_to_word = token_list_en, derivative_change_rate = derivative_change_rate, clip_max = clip_max )
# 全ステップ数
num_global_steps = len( ob_train ) * epoch_num
print( "num_global_steps:", num_global_steps )
num_warmup_steps = num_global_steps * 0.1
print( "num_warmup_steps:", num_warmup_steps )
derivative_change_step = num_global_steps * derivative_change_rate
print( "derivative_change_step:", derivative_change_step )
#スケジューラーの定義
scheduler = get_linear_schedule_with_warmup( optimizer, num_warmup_steps, num_global_steps )
use_param_file = False
PATH = "./model_NTT_auto_curr3.pt"
print_coef = 1000
save_file_coef = 2000

num_global_steps: 299694
num_warmup_steps: 29969.4
derivative_change_step: 2996.94


In [25]:
#seed = 0

#random.seed(seed)
#np.random.seed(seed)
#torch.manual_seed(seed)
#torch.backends.cudnn.benchmark = False
#torch.backends.cudnn.deterministic = True

text = torch.randint( 0, enc_vocab_size, size=(8, 100 ))
dec_input = torch.randint( 0, dec_vocab_size, size=(1,120))

seq_len_src = text.shape[1]
seq_len_tgt = dec_input.shape[1]

mask_tgt = nn.Transformer.generate_square_subsequent_mask( seq_len_tgt, dtype=bool ).to(device)
mask_src = torch.zeros((seq_len_src, seq_len_src), device=device).type(torch.bool)

padding_mask_src = (text == idx_list['<pad>'])
padding_mask_tgt = (dec_input == idx_list_en['<pad>'])

weights = OrderedDict(model.named_parameters())

model.eval()
outputs1 = model( text.to(device), dec_input.to(device) )
#outputs1 = model.encoder( text.to(device), None, None )
#outputs1 = model.encoder( text.to(device), dec_input None, None )
print( "outputs1 size:", outputs1.size())
print( outputs1[0][0][0])

#for name in weights:
#    print( name )
model.eval()
outputs2 = model.adaptation( text.to(device), dec_input.to(device), weights )
#outputs2 = model.fencoder( text.to(device), None, None,  weights )
print( "outputs2 size:", outputs2.size() )
print( outputs2[0][0][0])

outputs1 size: torch.Size([8, 120, 20556])
tensor(-1.7009, device='cuda:0', grad_fn=<SelectBackward0>)
outputs2 size: torch.Size([8, 120, 20556])
tensor(-1.7009, device='cuda:0', grad_fn=<SelectBackward0>)


In [26]:
print( "Train")
print( "use_param_file:", use_param_file )
print( "PATH:", PATH )
print( "epoch:", epoch_num )
print( "num_global_steps:",num_global_steps)
print( "num_warmup_steps:", num_warmup_steps)
print( "num_ob_val:", num_ob_val )
print( "print_coef:", print_coef )
print( "save_file_coef:", save_file_coef )
print( "derivative_change_step:", derivative_change_step )
      
if os.path.isfile(PATH) and use_param_file:
    print( device )
    if device != torch.device("cpu"):
        checkpoint = torch.load(PATH)
    else:
        checkpoint = torch.load(PATH, map_location=torch.device('cpu'))

    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    # optimizerのstateを現在のdeviceに移す。これをしないと、保存前後でdeviceの不整合が起こる可能性がある。
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.to(device)
    new_epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    tra_global_step = checkpoint['tra_global_step']
    val_global_step = checkpoint['val_global_step']

history_train = { "step": [], "train_loss": [], "train_wer": []}
history_val = { "step": [], "val_loss": [], "val_wer": [] }

n = 0
train_loss = 0
val_loss = 0

f_train = open( "train3.log", mode="w", encoding = "UTF-8" )
f_val = open( "val3.log", mode="w", encoding = "UTF-8" )

if not use_param_file:
    new_epoch = 0
    tra_global_step = 0
    val_global_step = 0

for epoch in range(new_epoch, epoch_num):
    db_train = create_batch_of_tasks( ob_train, is_shuffle = True, outer_batch_size = len( ob_train ) )
    model.train()
    train_loss = 0
    train_wer = 0
    n = 0
    for i, train_task in enumerate(db_train):
        tra_global_step += 1
        #print( "tra_global_step:", tra_global_step )
        progress_rate = tra_global_step / num_global_steps
        if tra_global_step % print_coef == 0:
            display = True
            train_loss = 0
            train_wer = 0
            n = 0
        else:
            display = False
        if display:
            print("\n-----------------Train Mode-----------------\n")
        lr_inner = optimizer.param_groups[0]["lr"]
        if display:
            print("lr:", lr_inner )
        model.train()
        loss, wer = adaptation( batch = train_task, train_step = 5, train = True, lr1 = lr_inner, display = display, 
                               progress_rate = progress_rate)
        scheduler.step()
        train_loss += loss
        train_wer += wer
        n += 1
        history_train["step"].append( tra_global_step )
        history_train["train_loss"].append( train_loss / n )
        history_train["train_wer"].append( train_wer / n )
        if display:
            print(f"epoch:{epoch+1}  Step:{tra_global_step}  loss:{train_loss/n:.10f}  WER:{ train_wer / n:.10f}")
        if tra_global_step % save_file_coef == 0:
            PATH = './model_NTT_auto_curr3.pt'
            torch.save({'epoch': epoch,
                'tra_global_step': tra_global_step,
                'val_global_step': val_global_step, 
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': loss,},
               PATH)
            with open("history_train_NTT_auto_curr3.pkl", "wb") as f:
                pickle.dump( history_train, f )

        f_train.write( "Epoch: " + str(epoch) + ", Step: " + str(tra_global_step) + ", Loss: " + str(train_loss/n) + ", WER: " + str(train_wer / n ) + "\n" )
        f_train.flush()

        val_loss = 0
        val_wer = 0
        val_n = 0
        display_taihi = display
        #if tra_global_step % 100 == 0:
        if display:
            print("\n-----------------Validation Mode-----------------\n")
            for i_val, val_task in enumerate( db_val ):
                if i_val == 0:
                    display = True
                else:
                    display = False
                val_global_step += 1
                #print( "val_global_step:", val_global_step )
                lr_inner = optimizer.param_groups[0]["lr"]
                loss, wer = adaptation( batch = val_task, train_step = 5, train = False, lr1 = lr_inner, display = display, 
                               progress_rate = progress_rate)
                val_loss += loss
                val_wer += wer
                val_n += 1
                history_val["step"].append( val_global_step / num_ob_val * print_coef )
                history_val["val_loss"].append( val_loss / val_n )
                history_val["val_wer"].append( val_wer / val_n )
            print(f"epoch:{epoch+1}  Step:{val_global_step}  loss:{val_loss/val_n:.10f}  WER:{ val_wer / val_n:.10f}")
            f_val.write( "Epoch: " + str(epoch) + ", Step: " + str(val_global_step)  + ", Loss: " + str(val_loss/val_n) + ", WER: " + str(val_wer / val_n ) + "\n" ) 
            f_val.flush()
        display = display_taihi
        if tra_global_step % save_file_coef == 0:
            with open("history_val_NTT_auto_curr3.pkl", "wb") as f:
                pickle.dump( history_val, f )

f_train.close()
f_val.close()

Train
use_param_file: False
PATH: ./model_NTT_auto_curr3.pt
epoch: 3
num_global_steps: 299694
num_warmup_steps: 29969.4
num_ob_val: 10
print_coef: 1000
save_file_coef: 2000
derivative_change_step: 2996.94

-----------------Train Mode-----------------

lr: 3.3334000680694313e-06
----Task 0 ----
Inner Loss:  6.312644004821777
train pred: the the the the the the the the of the the the to the the of the of the the of the the of the of the the to the the the the the <eos>
train targ: the final version of the draft agenda as drawn up by the conference of presidents at its meeting of thursday 10 may pursuant to rule 110 of the rules of procedure has been distributed <eos>
train pred: i the the the the of and and of and the of of the of and the of and and of of and and the the and of and of and and of and and of of the and of of and the and and of the and of the and and and of the the and and and of <eos>
train targ: (pt) in general terms the report addresses the issue of the implementation of

KeyboardInterrupt: 